<a href="https://colab.research.google.com/github/aborbala/tree-canopy/blob/main/01_03_DOP_slicing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

"""
Vegetation Mask Creation from Multispectral Imagery

A geospatial image processing workflow that creates vegetation masks from 
multispectral raster data using NDVI (Normalized Difference Vegetation Index) 
classification and morphological filtering.

Overview:
    This pipeline processes true color orthophoto (DOP) imagery with near-infrared
    bands to identify and extract vegetation areas in Berlin. The workflow calculates
    NDVI values, classifies vegetation density, applies smoothing filters, and
    converts the result to vector geometries with spatial buffers.

Main Sections:
    
    1. Setup & Configuration
       - Mount Google Drive and install dependencies
       - Configure file paths for input DOP imagery and output directory
       - Set up area of interest (AOI) parameters
    
    2. Data Exploration & Metadata
       - Read multispectral raster metadata (CRS, bounds, bands)
       - Extract NIR (Band 1) and Red (Band 2) spectral data
       - Compute per-band statistics and percentiles
    
    3. NDVI Calculation
       - Compute Normalized Difference Vegetation Index: (NIR - Red) / (NIR + Red)
       - Analyze NDVI distribution with statistics and histogram
       - Visualize NDVI values as raster imagery
    
    4. NDVI Classification
       - Classify pixels into vegetation density classes:
         * Water: NDVI < 0
         * Bare soil: 0 ≤ NDVI < 0.2
         * Sparse vegetation: 0.2 ≤ NDVI < 0.6
         * Dense vegetation: NDVI ≥ 0.6
       - Export classified raster with categorical colormap
       - Visualize classification results
    
    5. Binary Vegetation Mask Creation
       - Convert NDVI to binary mask using threshold (NDVI > 0.16)
       - Separate vegetation from non-vegetation pixels
       - Export unsmoothed binary mask to GeoTIFF
    
    6. Morphological Filtering & Smoothing
       - Apply median filter (5×5 kernel) to reduce noise
       - Re-threshold filtered result to binary
       - Smooth jagged vegetation boundaries
    
    7. Vector Conversion & Buffering
       - Convert raster mask to vector polygons (GeoDataFrame)
       - Apply 1-meter spatial buffer around vegetation patches
       - Simplify geometry with 1-meter tolerance
       - Dissolve overlapping geometries into single features
       - Export final vector mask as GeoPackage (GPKG)

Key Data:
    - Input: True Color Orthophoto (DOP) with NIR bands (4 bands total)
    - Processing: NDVI calculation, threshold-based classification, morphological ops
    - Output:
        * NDVI raster (float32)
        * Classified NDVI raster (categorical)
        * Binary vegetation mask (uint8) - unsmoothed
        * Binary vegetation mask (uint8) - smoothed
        * Vector vegetation mask with buffers (GPKG)

CRS:
    - EPSG:25833 (UTM Zone 33N) - Standard for Berlin/Germany region
    - CRS automatically detected from input raster

Input Format:
    - GeoTIFF raster with 4 bands (NIR, Red, Green, Blue)
    - Band 1: Near-infrared (NIR)
    - Band 2: Red
    - Bands 3-4: Green and Blue (metadata)

Output Locations:
    - {aoi_code}/vegetation_mask/ndvi.tif - Raw NDVI values
    - {aoi_code}/vegetation_mask/ndvi_classified.tif - Classified NDVI (4 classes)
    - {aoi_code}/vegetation_mask/veg_mask_unsmoothed.tif - Binary mask (no filtering)
    - {aoi_code}/vegetation_mask/veg_mask_smoothed.tif - Binary mask (median filtered)
    - {aoi_code}/vegetation_mask/veg_mask_buffered.gpkg - Vector polygons with buffers

Parameters:
    - NDVI vegetation threshold: 0.16
    - Median filter kernel size: 5×5 pixels
    - Vector buffer distance: 1 meter
    - Geometry simplification tolerance: 1 meter
"""

In [2]:
! pip install --upgrade rasterio laspy -Uqq ipdb

In [3]:
import rasterio
import numpy as np
import os
import laspy
from rasterio.windows import Window
from rasterio.plot import show
import ipdb
from shapely.geometry import box

In [4]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load helper functions
#!ls /content/drive/MyDrive/utils/*.py
#!cat '/content/drive/My Drive/utils/utils.py'
#import sys
#sys.path.append('/content/drive/MyDrive/utils/')
#import utils
#utils.test()

In [5]:
def get_las_bbox(file_path):
    """
    Get the bounding box coordinates from a LAS file.

    Args:
        file_path (str): Path to the LAS file.

    Returns:
        tuple: (min_x, min_y, max_x, max_y) coordinates of the bounding box.
    """
    las = laspy.read(file_path)
    min_x = las.header.min[0]
    min_y = las.header.min[1]
    max_x = las.header.max[0]
    max_y = las.header.max[1]
    return min_x, min_y, max_x, max_y

In [6]:
def convert_to_rgb_tiff(input_image_path, output_image_path, num_bands=3):
    """
    Convert an input image to an RGB TIFF imagee.

    Args:
        input_image_path (str): Path to the input image file.
        output_image_path (str): Path to the output RGB TIFF file.
        num_bands (int, optional): Number of bands to include in the output image. Defaults to 3.

    Returns:
        None
    """
    with rasterio.open(input_image_path) as src:
        profile = src.profile
        image_data = src.read()

    # Select the first three bands
    rgb_image_data = image_data[0:3, :, :]

    # Update the profile for the output image
    profile.update(count=3)

    # Write the RGB image to a new TIFF file
    with rasterio.open(output_image_path, 'w', **profile) as dst:
        dst.write(rgb_image_data)

In [7]:
def ensure_rgb_tiff(input_image_path, output_image_path=None, num_bands=3):
    """
    Ensure the input image is an RGB TIFF with the specified number of bands for PIL package.

    Args:
        input_image_path (str): Path to the input image file.
        output_image_path (str, optional): Path to the output RGB TIFF file. Defaults to None.
        num_bands (int, optional): Number of bands to include in the output image. Defaults to 3.

    Returns:
        str: Path to the output RGB TIFF file if conversion is needed, otherwise the input image path.
    """
    with rasterio.open(input_image_path) as src:
        image_data = src.read()

    # Check if the input image has more bands than the specified 'num_bands'
    if image_data.shape[0] > num_bands:
        if output_image_path is None:
            base, ext = os.path.splitext(input_image_path)
            output_image_path = f"{base}_rgb{ext}"

        # Call the 'convert_to_rgb_tiff' function to convert the input image
        convert_to_rgb_tiff(input_image_path, output_image_path, num_bands)
        return output_image_path
    else:
        return input_image_path

In [8]:
def crop_tif_by_bbox(tif_file_path, bbox):
    """
    Crop a TIFF image using a bounding box (LAS) to ensure compatibility.

    Args:
        tif_file_path (str): Path to the TIFF image file.
        bbox (tuple): Bounding box coordinates (min_x, min_y, max_x, max_y).

    Returns:
        tuple: Cropped image data and updated profile.
    """
    with rasterio.open(tif_file_path) as src:
        min_x, min_y, max_x, max_y = bbox
        window = src.window(min_x, min_y, max_x, max_y)
        transform = src.window_transform(window)
        profile = src.profile
        profile.update({
            'height': window.height,
            'width': window.width,
            'transform': transform
        })
        data = src.read(window=window)

        # Ensure the data is in RGB format
        if data.shape[0] > 3:
            data = data[0:3, :, :]
            profile.update(count=3)

    return data, profile


In [9]:
tile_records = []

In [10]:
def slice_image(input_data, input_profile, output_folder, tilename, tile_size=100, overlap=20):
    """
    Slices an image into overlapping tiles using a sliding window approach.

    Args:
        input_data (numpy.ndarray): Image data array.
        input_profile (dict): Raster metadata.
        output_folder (str): Folder to save tiles.
        tilename (str): Base name for tiles.
        tile_size (int, optional): Size of the tile in meters. Defaults to 100.
        overlap (int, optional): Overlap in meters. Defaults to 20.

    Returns:
        None
    """
    global tile_records  # Store tile bounding boxes globally

    rows, cols = input_data.shape[1], input_data.shape[2]
    transform = input_profile['transform']
    profile = input_profile.copy()

    # Convert tile size and overlap to pixels
    pixel_size_x = transform.a
    pixel_size_y = -transform.e

    tile_size_x = int(tile_size / pixel_size_x)
    tile_size_y = int(tile_size / pixel_size_y)

    step_x = tile_size_x - int(overlap / pixel_size_x)
    step_y = tile_size_y - int(overlap / pixel_size_y)

    tile_id = 0

    row_positions = list(range(0, rows, step_y))
    if row_positions[-1] + tile_size_y < rows:  # Ensure last row is covered
        row_positions.append(rows - tile_size_y)

    col_positions = list(range(0, cols, step_x))
    if col_positions[-1] + tile_size_x < cols:  # Ensure last column is covered
        col_positions.append(cols - tile_size_x)

    for row_start in row_positions:
        for col_start in col_positions:
            row_end = min(row_start + tile_size_y, rows)
            col_end = min(col_start + tile_size_x, cols)

            actual_tile_size_y = row_end - row_start
            actual_tile_size_x = col_end - col_start

            window = Window(col_start, row_start, actual_tile_size_x, actual_tile_size_y)
            window_transform = rasterio.windows.transform(window, transform)

            profile.update({
                'height': actual_tile_size_y,
                'width': actual_tile_size_x,
                'transform': window_transform
            })

            output_data = input_data[:, row_start:row_end, col_start:col_end]

            output_file_path = os.path.join(output_folder, f"{tilename}_{tile_id}.tif")
            print(f"Saving: {output_file_path}, Row {row_start}-{row_end}, Col {col_start}-{col_end}")

            with rasterio.open(output_file_path, 'w', **profile) as dst:
                dst.write(output_data)

            # Extract BBOX
            x_min, y_max = window_transform * (0, 0)  # Top-left corner
            x_max, y_min = window_transform * (actual_tile_size_x, actual_tile_size_y)  # Bottom-right corner

            tile_records.append({
                "tile_name": f"{tilename}_{tile_id}",
                "geometry": box(x_min, y_min, x_max, y_max)  # Bounding box
            })

            tile_id += 1

In [15]:
# Define parameters
TILE_SIZE = 100  # Tile size in meters
time_suffix = '2020S'

# aoi_code = '386_5818' # training data
aoi_code = '384_5816' # test data


In [28]:
base_drive = '/content/drive/MyDrive/masterthesis/data'

# Path to the folder containing LAS files without buildings
#las_folder_path = '/content/drive/MyDrive/data/382_5826_1/LAS_no_buildings'
#las_folder_path = '/content/drive/MyDrive/data/400_5816/LAS_no_buildings'
las_folder_path = f'{base_drive}/{aoi_code}/LAS_no_buildings_veg_mask'

## Summer 2020
#tif_file_path = '/content/drive/MyDrive/data/382_5826_1/DOP/truedop20rgb_382_5826_2_be_2020.tif'
#sliced_output_folder_path = '/content/drive/MyDrive/data/382_5826_1/sliced_output_2020S'

# Paths to the TIFF files for different time periods and locations
## Winter 02.2021 382_5826
#tif_file_path = '/content/drive/MyDrive/data/382_5826_1/DOP/dop20rgbi_33_382_5826_2_be_2022.tif'
#sliced_output_folder_path = '/content/drive/MyDrive/data/382_5826_1/sliced_output_2021W'

## Summer 2020 386_5818
tif_file_path = f'{base_drive}/{aoi_code}/DOP/truedop20rgb_{aoi_code}_2_be_2020.tif'
sliced_output_folder_path = f'{base_drive}/{aoi_code}/sliced_imgs_{time_suffix}'

useMasking = False

mask_file_path = f'{base_drive}/{aoi_code}/cadaster_mask.gpkg'
masked_tif_path = f'{base_drive}/{aoi_code}/DOP/dop20rgb_masked.tif'
masked_sliced_output_folder_path = f'{base_drive}/{aoi_code}/masked_sliced_imgs_{time_suffix}'

## Winter 2021 386_5818
#tif_file_path = '/content/drive/MyDrive/data/386_5818/DOP/dop20rgb_386_5818_2_be_2021.tif'
#sliced_output_folder_path = '/content/drive/MyDrive/data/386_5818/sliced_imgs_2021W'

# Define the output path for tile metadata
tile_metadata_path = f'{base_drive}/{aoi_code}/tiles.geojson'

os.makedirs(sliced_output_folder_path, exist_ok=True)

In [29]:
from rasterio.mask import mask
from shapely.geometry import mapping
import geopandas as gpd
import numpy as np

if (useMasking):
  mask_gdf = gpd.read_file(mask_file_path)
  mask_gdf.plot()
  # Open the raster file
  with rasterio.open(tif_file_path) as src:
    raster_crs = src.crs  # Get CRS of raster

    # Reproject mask to match raster CRS
    if mask_gdf.crs != raster_crs:
        mask_gdf = mask_gdf.to_crs(raster_crs)

    # Convert geometry to JSON-like mapping
    mask_shapes = [mapping(geom) for geom in mask_gdf.geometry]

    # Apply the mask: set intersecting areas to NoData
    masked_raster, masked_transform = mask(src, mask_shapes, crop=False)
    nodata_value = src.nodata if src.nodata is not None else 255

    mask_indices = masked_raster == 0
    masked_raster[:, mask_indices[0]] = nodata_value

    # Copy metadata and update it for the new file
    out_meta = src.meta.copy()
    out_meta.update({"transform": masked_transform, "nodata": nodata_value})

    # Save the masked raster
    with rasterio.open(masked_tif_path, "w", **out_meta) as dest:
        dest.write(masked_raster)

In [25]:
import matplotlib.pyplot as plt

if (useMasking):
  # 🔹 **Plot the masked raster**
  fig, ax = plt.subplots(figsize=(10, 10))
  ax.imshow(np.moveaxis(masked_raster[:3], 0, -1))  # Move channels to last axis for RGB display
  ax.set_title("Masked Raster Preview")
  ax.axis("off")
  plt.show()

In [30]:
# Iterate over each file in the LAS folder
for file in os.listdir(las_folder_path):
    print(file)
    if file.endswith('.las'):
        las_file_path = os.path.join(las_folder_path, file)
        bbox = get_las_bbox(las_file_path)
        print(las_file_path)

        if (useMasking):
          # Call the crop_tif_by_bbox function and get the cropped data and profile
          cropped_data, cropped_profile = crop_tif_by_bbox(masked_tif_path, bbox)
        else:
          cropped_data, cropped_profile = crop_tif_by_bbox(tif_file_path, bbox)

        # Call the slice_image function for the cropped data and profile
        tilename = os.path.splitext(file)[0]
        slice_image(cropped_data, cropped_profile, sliced_output_folder_path, tilename)

3dm_33_384_5816_1_be_nobuild.las
/content/drive/MyDrive/masterthesis/data/384_5816/LAS_no_buildings_veg_mask/3dm_33_384_5816_1_be_nobuild.las
Saving: /content/drive/MyDrive/masterthesis/data/384_5816/sliced_imgs_2020S/3dm_33_384_5816_1_be_nobuild_0.tif, Row 0-500, Col 0-500
Saving: /content/drive/MyDrive/masterthesis/data/384_5816/sliced_imgs_2020S/3dm_33_384_5816_1_be_nobuild_1.tif, Row 0-500, Col 400-900
Saving: /content/drive/MyDrive/masterthesis/data/384_5816/sliced_imgs_2020S/3dm_33_384_5816_1_be_nobuild_2.tif, Row 0-500, Col 800-1300
Saving: /content/drive/MyDrive/masterthesis/data/384_5816/sliced_imgs_2020S/3dm_33_384_5816_1_be_nobuild_3.tif, Row 0-500, Col 1200-1700
Saving: /content/drive/MyDrive/masterthesis/data/384_5816/sliced_imgs_2020S/3dm_33_384_5816_1_be_nobuild_4.tif, Row 0-500, Col 1600-2100
Saving: /content/drive/MyDrive/masterthesis/data/384_5816/sliced_imgs_2020S/3dm_33_384_5816_1_be_nobuild_5.tif, Row 0-500, Col 2000-2500
Saving: /content/drive/MyDrive/masterthesis

In [31]:
# Convert tile records into a GeoDataFrame
tiles_gdf = gpd.GeoDataFrame(tile_records, geometry="geometry", crs="EPSG:25833")  # Modify EPSG if needed

# Save tile bounding boxes
tiles_gdf.to_file(tile_metadata_path, driver="GeoJSON")

In [ ]:
# Slice the tile: 33_382_5826_1 (1km x 1km)
#input_image_path = "/content/drive/MyDrive/data/382_5826_1/dop20rgbi_33_382_5826_2_be_2021_1.tif" # 2021W
#rgb_image_path = '/content/drive/MyDrive/data/382_5826_1/dop20rgbi_33_382_5826_2_be_2021_1_rgb.tif'

In [ ]:
# If the input image has more bands than 'num_bands', the function returns the path to the new RGB image.
# Otherwise, it returns the input image path.
#result_image_path = ensure_rgb_tiff(input_image_path, rgb_image_path)

In [ ]:
#utils.sliceImg(result_image_path, "/content/drive/MyDrive/data/382_5826_1/sliced_imgs_2021W/", "382_5826")

## Now go to R and extract the tree crowns for each tile